#Machine Learning and Remote Sensing for Wildfire and Spatial Risk Management

## Data aquisiton

Every machine learning algorithm relies havily on data. Remote sensing data is valuble resource for monitoring earth surface and recognizing varieties of phenomenons -  states, eventss, changes

##Data -> information -> knowledge
Data - raw values
Information - values associated with metadata (such as time and place of aquisition, sensor, measuring unit ...)
Knowledge - detection of circumstances associated with value (Such as wildfire or urban heat island)

In [3]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------- ----------------------------- 2.1/8.1 MB 10.7 MB/s eta 0:00:01
   ---------------------- ----------------- 4.5/8.1 MB 11.7 MB/s eta 0:00:01
   ----------------------------------- ---- 7.1/8.1 MB 11.5 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 11.3 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 11.5 MB/s eta 0:00:00
Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl (71 kB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------------------------------- ---- 2.4/2.7 MB 12.3 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 11.1 MB/s eta 0:00:

In [ ]:
!pip install openeo xarray netCDF4 h5netcdf geopandas matplotlib
 

  Using cached netCDF4-1.7.2-cp312-cp312-win_amd64.whl.metadata (1.8 kB)
  Using cached numpy-2.3.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached xarray-2025.1.1-py3-none-any.whl.metadata (11 kB)
  Using cached pystac-1.11.0-py3-none-any.whl.metadata (4.5 kB)
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached oschmod-0.3.12-py2.py3-none-any.whl.metadata (10.0 kB)
  Using cached cftime-1.6.4.post1-cp312-cp312-win_amd64.whl.metadata (8.9 kB)
  Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached xarray-2025.1.1-py3-none-any.whl (1.2 MB)
Using cached pystac-1.11.0-py3-none-any.whl (183 kB)
Using cached netCDF4-1.7.2-cp312-cp312-win_amd64.whl (7.0 MB)
Using cached Deprecated-1.2.18-py2.py3-none-any.whl (10.0 kB)
Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl (38 kB)
Using cached numpy-2.3.1-cp312-cp312-win_amd64.whl (12.7 MB)
Using cached oschmod-0.3

In [4]:
import openeo
import xarray
import matplotlib.pyplot as plt

#OpenEO


In [5]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [6]:
s2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=("2024-07-15", "2024-08-15"),
    spatial_extent={
        "west": 15.74,
        "south": 43.7,
        "east": 16.00,
        "north": 43.93,
        "crs": "EPSG:4326",
    },
    #bands=["B04", "B03", "B02", "SCL"],
    bands=['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12'],
    max_cloud_cover=50,
)
s2_cube.download("fire_krka.nc")

In [ ]:
ds = xarray.load_dataset("fire_krka.nc")
# Convert xarray DataSet to a (bands, t, x, y) DataArray
data = ds[["B04", "B03", "B02"]].to_array(dim="bands")
data

Compare images from 3 timestamps - before, during and after fire
Burned area is visible on the third image

In [ ]:
data[{"t": 0}].plot.imshow(vmin=0, vmax=2000)


In [ ]:

fig, axes = plt.subplots(ncols=3, figsize=(15, 3), dpi=90, sharey=True)
data[{"t": 0}].plot.imshow(vmin=0, vmax=2000, ax=axes[0])
data[{"t": 2}].plot.imshow(vmin=0, vmax=2000, ax=axes[1]);
data[{"t": 3}].plot.imshow(vmin=0, vmax=2000, ax=axes[2]);

<h2> Explore netCDF</h2>

In [ ]:
import netCDF4

# Open the NetCDF file
fp = 'fire_krka.nc'
nc = netCDF4.Dataset(fp)

# List all variables
print(nc.variables.keys())

# List all dimensions
print(nc.dimensions.keys())

# Access a variable (e.g., 'Temp')
temp = nc.variables['t'][:]
print(temp)
# Print CRS
print(nc.variables['crs'])
# Close the file when done
nc.close()

<h2> Load Burned areas from effis</h2>

In [ ]:
import geopandas as gpd
import pandas as pd
# Load shapefile
gdf1 = gpd.read_file("effis_layer/modis.ba.poly.shp")

import pandas as pd
#df = pd.DataFrame(gdf.drop(columns='geometry'))
# Convert date column to datetime
gdf1['date'] = pd.to_datetime(gdf1['FIREDATE'],format='mixed')

# Define filter criteria
countries_to_keep = ['HR' ]
start_date = '2024-01-01'
end_date = '2024-12-31'

# Filter by country and date range
gdf = gdf1[
    (gdf1['COUNTRY'].isin(countries_to_keep)) &
    (gdf1['date'] >= start_date) &
    (gdf1['date'] <= end_date)
]
 

Create bounding boxes around every fire

In [ ]:
# Extract bounding box coordinates (west, south, east, north) for each geometry
gdf['bbox'] = gdf.geometry.apply(lambda geom: geom.bounds)  # bounds = (minx, miny, maxx, maxy)

# Split bounds tuple into separate columns
gdf[['west', 'south', 'east', 'north']] = gdf['bbox'].apply(pd.Series)
 

# Example: 10 days before and 10 days after fire date
gdf['start_date_before'] = gdf['date'] - pd.Timedelta(days=10)
gdf['end_date_before'] = gdf['date'] - pd.Timedelta(days=1)

gdf['start_date_after'] = gdf['date'] + pd.Timedelta(days=2)
gdf['end_date_after'] = gdf['date'] + pd.Timedelta(days=10)
 

In [ ]:
import openeo
import xarray
import matplotlib.pyplot as plt
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection
 
connection.authenticate_oidc()

In [ ]:
Downlaod EO data 

In [ ]:
'''
from openeo.rest.connection import OpenEoApiError
import os
import time
all_bands = [
 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'WVP', 'AOT', 'SCL', 'sunAzimuthAngles', 'sunZenithAngles', 'viewAzimuthMean', 'viewZenithMean'
]
for idx, row in gdf.iterrows():
    id = row['id']
    file_before = f"fire_{id}_before.nc"
    file_after = f"fire_{id}_after.nc"
    spatial_extent = {
        "west": row['west'],
        "south": row['south'],
        "east": row['east'],
        "north": row['north'],
        "crs": "EPSG:4326",
    }
    if os.path.exists(file_after):
        print(f"File {file_after} already exists, skipping download.")
    else:
    # Define date ranges as strings
        print(f"downloading {file_before} ." )
        before_start = row['start_date_before'].strftime('%Y-%m-%d')
        before_end = row['end_date_before'].strftime('%Y-%m-%d')
        after_start = row['start_date_after'].strftime('%Y-%m-%d')
        after_end = row['end_date_after'].strftime('%Y-%m-%d')
        
        # Download before fire
        try:
            s2_cube_before = connection.load_collection(
                "SENTINEL2_L2A",
                temporal_extent=(before_start, before_end),
                spatial_extent=spatial_extent,
                bands=all_bands,
                
            ).reduce_dimension(dimension="t", reducer="last")
            s2_cube_before.download(file_before)
            print(f"  {file_before} . downloaded , sleeping" )
            time.sleep(5)
        except OpenEoApiError as e:
            if "NoDataAvailable" in str(e):
                print(f"No data available for fire_{id}_before.nc, skipping download.")
            else:
                raise  # re-raise other errors
        
        # Download after fire

        print(f"downloading {file_after}  " )
        try:
            s2_cube_after = connection.load_collection(
                "SENTINEL2_L2A",
                temporal_extent=(after_start, after_end),
                spatial_extent=spatial_extent,
                bands=all_bands,
               
            ).reduce_dimension(dimension="t", reducer="first")
            s2_cube_after.download(file_after)
            print(f"  {file_before} . downloaded , sleeping" )

            time.sleep(5)
        except OpenEoApiError as e:
            if "NoDataAvailable" in str(e):
                print(f"No data available for fire_{id}_after.nc, skipping download.")
            else:
                raise
'''